In [1]:
import pandas as pd
from tqdm import tqdm
import datetime
from collections import defaultdict
import argparse

In [21]:
sp = 0
ep = 100

In [4]:
label_df = pd.read_csv('../data/label.csv')
raw_labevents_df = pd.read_csv('../data/raw_labevents.csv')
raw_patientsinfo_df = pd.read_csv('../data/raw_patientsinfo.csv')
raw_procedureevents_df = pd.read_csv('../data/raw_procedureevents.csv')
raw_chatted_event_df = pd.read_csv('../data/raw_chart_event.csv')

In [8]:
raw_chatted_event_df

,Unnamed: 0,subject_id,hadm_id,itemid,label,charttime,storetime,value
0,0,16467047,25070073,220045,Heart Rate,2189-10-12 18:00:00,2189-10-12 19:05:00,110
1,1,16467047,25070073,220050,Arterial Blood Pressure systolic,2189-10-12 18:00:00,2189-10-12 19:05:00,97
2,2,16467047,25070073,220179,Non Invasive Blood Pressure systolic,2189-10-12 18:00:00,2189-10-12 19:05:00,53
3,3,16467047,25070073,220210,Respiratory Rate,2189-10-12 18:00:00,2189-10-12 19:05:00,24
4,4,16467047,25070073,220045,Heart Rate,2189-10-12 19:00:00,2189-10-12 19:05:00,121
...,...,...,...,...,...,...,...,...
19456672,19456672,16467047,25070073,224027,Skin Temperature,2189-10-12 16:54:00,2189-10-12 16:55:00,Warm
19456673,19456673,16467047,25070073,220179,Non Invasive Blood Pressure systolic,2189-10-12 17:30:00,2189-10-12 19:05:00,75
19456674,19456674,16467047,25070073,220045,Heart Rate,2189-10-12 17:32:00,2189-10-12 19:05:00,108
19456675,19456675,16467047,25070073,220050,Arterial Blood Pressure systolic,2189-10-12 17:32:00,2189-10-12 19:05:00,81


In [5]:
main_df = label_df

In [6]:
main_df = main_df.merge(raw_patientsinfo_df, on='subject_id', how='left')

In [20]:
rows = list()

for index, row in tqdm(list(main_df.iterrows())[sp:ep]):
    subject_id = row['subject_id']
    hadm_id = row['hadm_id']
    clabsi_label = row['clabsi']
    gender = row["gender"]
    age = row["age"]
    
    Invasive_table = raw_procedureevents_df[(raw_procedureevents_df["subject_id"] == subject_id) & (raw_procedureevents_df["hadm_id"] == hadm_id)]
    starttimes = [datetime.datetime.strptime(t_str, "%Y-%m-%d %H:%M:%S") for t_str in Invasive_table["starttime"]]
    endtimes = [datetime.datetime.strptime(t_str, "%Y-%m-%d %H:%M:%S") for t_str in Invasive_table["endtime"]]

    # Start time and End time
    start_time = sorted(starttimes)[0]
    end_time = sorted(endtimes)[-1] + datetime.timedelta(days=2)

    feature = dict()

    for item_id in [228242, 224690, 220210, 224689, 220179, 225309, 223762, 227243, 220045, 224167, 223761, 220050]:
        s_time = start_time
        e_time = start_time + datetime.timedelta(days=1)

        event_result = raw_chatted_event_df[(raw_chatted_event_df["subject_id"] == subject_id) & (raw_chatted_event_df["hadm_id"] == hadm_id) & (raw_chatted_event_df["itemid"] == item_id)]
        event_list = list()
        for index, event_row in event_result.iterrows():
            event_list += [{
                "charttime": datetime.datetime.strptime(event_row["charttime"], "%Y-%m-%d %H:%M:%S"),
                "value": event_row["value"]
            }]

        time_point = 0
        while e_time <= end_time:
            if time_point not in feature:
                feature[time_point] = dict()

            s_time += datetime.timedelta(days=1)
            e_time += datetime.timedelta(days=1)

            temp_list = list()
            for event in event_list:
                if event["charttime"] >= s_time and event["charttime"] < e_time:
                    temp_list += [float(event["value"])]

            feature[time_point][item_id]  = {
                "label": item_id,
                "time_point": time_point,
                "max": max(temp_list) if temp_list else None,
                "min": min(temp_list) if temp_list else None, 
                "average": (sum(temp_list) / len(temp_list)) if temp_list else None, 
            }
            time_point += 1    
    
    for timepoint in feature:
        rows += [{
            "subject_id": subject_id,
            "hadm_id": hadm_id,
            "timepoint": timepoint,
            "clabsi": clabsi_label,
            "gender": gender,
            "age": age,
            "HR_max": feature[timepoint][220045]["max"],
            "HR_min": feature[timepoint][220045]["min"],
            "PtS_max": feature[timepoint][228242]["max"],
            "PtS_min": feature[timepoint][228242]["min"],
            "Respiratory Rate (Total)_max": feature[timepoint][224690]["max"],
            "Respiratory Rate (Total)_min": feature[timepoint][224690]["min"],
            "Respiratory Rate_max": feature[timepoint][220210]["max"],
            "Respiratory Rate_min": feature[timepoint][220210]["min"],
            "Respiratory Rate (spontaneous)_max": feature[timepoint][224689]["max"],
            "Respiratory Rate (spontaneous)_min": feature[timepoint][224689]["min"],
            "NBPs_max": feature[timepoint][220179]["max"],
            "NBPs_min": feature[timepoint][220179]["min"],
            "ART BP Systolic_max": feature[timepoint][225309]["max"],
            "ART BP Systolic_min": feature[timepoint][225309]["min"],
            "Temperature C_max": feature[timepoint][223762]["max"],
            "Temperature C_min": feature[timepoint][223762]["min"],
            "Manual BPs R_max": feature[timepoint][227243]["max"],
            "Manual BPs R_min": feature[timepoint][227243]["min"],
            "Manual BPs L_max": feature[timepoint][224167]["max"],
            "Manual BPs L_min": feature[timepoint][224167]["min"],
            "Temperature F_max": feature[timepoint][223761]["max"],
            "Temperature F_min": feature[timepoint][223761]["min"],
            "ABPs_max": feature[timepoint][220050]["max"],
            "ABPs_min": feature[timepoint][220050]["min"],
        }]
df = pd.DataFrame(rows)

df.to_csv(f"../data/result_{sp}_{ep}.csv")

100%|██████████| 10/10 [00:07<00:00,  1.33it/s]


In [38]:
lab_df = pd.read_csv('../data/lab_data.csv')
chart_df = pd.read_csv('../data/chart_data.csv')

In [43]:
chart_df = chart_df.loc[:, ~chart_df.columns.str.contains('^Unnamed')]

In [46]:
chart_df.to_csv("../data/chart_data.csv")